Initializes notebook: inline plots, adds src/ to Python path, sets IMAGES_DIR to images1/, and prints the path.

In [ ]:
# Initial setup: paths and imports
%matplotlib inline
from pathlib import Path
import sys

ROOT = Path.cwd()
SRC_DIR = ROOT / 'src'
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))  # allow `import ...` from local src/

IMAGES_DIR = ROOT / 'images1'
print('Ready. Images path:', IMAGES_DIR)

Lists JPGs in IMAGES_DIR and displays the first one (RGB) with title and shape.

In [ ]:
# Preview a sample image (robust if setup cell wasn't run)
from pathlib import Path
from skimage import io
import matplotlib.pyplot as plt

IMAGES_DIR = globals().get('IMAGES_DIR', Path.cwd() / 'images1')

image_files = sorted(IMAGES_DIR.glob('*.jpg'))
assert image_files, f'No .jpg images found in {IMAGES_DIR}'

img_path = image_files[0]
img = io.imread(img_path)  # RGB

plt.imshow(img)
plt.title(img_path.name)
plt.axis('off')
print('Image shape:', img.shape)

Binarize the sample image: convert to grayscale, lightly blur, apply Otsu threshold, and show original vs binary.

In [ ]:
# Binarize image: grayscale -> blur -> Otsu threshold
from pathlib import Path
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import gaussian, threshold_otsu
import matplotlib.pyplot as plt

# reuse loaded image if present; otherwise load first jpg
if 'img' not in globals():
    IMAGES_DIR = globals().get('IMAGES_DIR', Path.cwd() / 'images1')
    img_path = sorted(IMAGES_DIR.glob('*.jpg'))[0]
    img = io.imread(img_path)

gray = rgb2gray(img)              # 0..1 float
blur = gaussian(gray, sigma=1.0)  # light denoise
t = threshold_otsu(blur)
bw = blur < t                     # ink is dark -> True

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(img);  axes[0].set_title('Original');  axes[0].axis('off')
axes[1].imshow(bw, cmap='gray'); axes[1].set_title(f'Binarized (t={t:.3f})'); axes[1].axis('off')
print('gray shape:', gray.shape, 'threshold:', t)

Clean the binary mask by removing small components (noise); visualize before vs. after.

In [ ]:
# Clean binary mask: remove tiny components (noise)
import matplotlib.pyplot as plt
from skimage.morphology import remove_small_objects

# fallbacks if earlier cells weren't run
if 'img' not in globals():
    from pathlib import Path
    from skimage import io
    IMAGES_DIR = globals().get('IMAGES_DIR', Path.cwd() / 'images1')
    img = io.imread(sorted(IMAGES_DIR.glob('*.jpg'))[0])
if 'bw' not in globals():
    from skimage.color import rgb2gray
    from skimage.filters import gaussian, threshold_otsu
    gray = rgb2gray(img)
    t = threshold_otsu(gaussian(gray, 1.0))
    bw = gray < t

clean = remove_small_objects(bw, min_size=150)  # adjust if needed

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(bw, cmap='gray');    axes[0].set_title('Binary');  axes[0].axis('off')
axes[1].imshow(clean, cmap='gray'); axes[1].set_title('Cleaned'); axes[1].axis('off')

Label connected components on the cleaned mask and draw bounding boxes over the original image.

In [ ]:
# Fix: compute boxes using .start/.stop from find_objects slices
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from scipy import ndimage as ndi
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import gaussian, threshold_otsu
from pathlib import Path

# fallbacks if earlier cells weren't run
if 'img' not in globals():
    IMAGES_DIR = globals().get('IMAGES_DIR', Path.cwd() / 'images1')
    img = io.imread(sorted(IMAGES_DIR.glob('*.jpg'))[0])
if 'clean' not in globals():
    gray = rgb2gray(img)
    blur = gaussian(gray, 1.0)
    clean = blur < threshold_otsu(blur)

min_size = 150

labels, num = ndi.label(clean)
slices = ndi.find_objects(labels)
areas = np.bincount(labels.ravel())  # area per label (index 0 is background)

boxes = []
for i, slc in enumerate(slices, start=1):  # labels 1..num
    if slc is None:
        continue
    r0, r1 = slc[0].start, slc[0].stop
    c0, c1 = slc[1].start, slc[1].stop
    if areas[i] < min_size:
        continue
    boxes.append((r0, c0, r1, c1))

boxes.sort(key=lambda b: b[1])  # left-to-right

fig, ax = plt.subplots(figsize=(8, 6))
ax.imshow(img)
for r0, c0, r1, c1 in boxes:
    ax.add_patch(Rectangle((c0, r0), c1 - c0, r1 - r0,
                           fill=False, edgecolor='lime', linewidth=2))
ax.set_title(f'Components (SciPy): {len(boxes)}')
ax.axis('off')
print('Detected components:', len(boxes))

Extract component crops, pad to square, resize to 28×28, and display them in reading order.

In [ ]:
# Extract crops from boxes, pad to square, resize to 28x28, show grid
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import rgb2gray
from skimage.transform import resize
from pathlib import Path

# fallbacks
if 'img' not in globals():
    from skimage import io
    IMAGES_DIR = globals().get('IMAGES_DIR', Path.cwd() / 'images1')
    img = io.imread(sorted(IMAGES_DIR.glob('*.jpg'))[0])
if 'gray' not in globals():
    gray = rgb2gray(img)

# if boxes missing, compute from clean mask quickly
if 'boxes' not in globals():
    from scipy import ndimage as ndi
    from skimage.filters import gaussian, threshold_otsu
    blur = gaussian(gray, 1.0)
    clean = blur < threshold_otsu(blur)
    labels, _ = ndi.label(clean)
    slices = ndi.find_objects(labels)
    areas = np.bincount(labels.ravel())
    min_size = 150
    boxes = []
    for i, slc in enumerate(slices, start=1):
        if slc is None or areas[i] < min_size:
            continue
        r0, r1 = slc[0].start, slc[0].stop
        c0, c1 = slc[1].start, slc[1].stop
        boxes.append((r0, c0, r1, c1))

# sort reading order: top-to-bottom then left-to-right
if boxes:
    med_h = np.median([b[2]-b[0] for b in boxes]) or 1
    row_h = max(1, int(0.6 * med_h))
    boxes = sorted(boxes, key=lambda b: (b[0] // row_h, b[1]))

def crop_norm(g, b, out=28, pad=2):
    r0, c0, r1, c1 = b
    crop = g[r0:r1, c0:c1]
    # trim empty margins using a simple threshold
    thr = np.percentile(crop, 80)  # paper is bright
    mask = crop < thr
    if mask.any():
        rr = np.where(mask.any(axis=1))[0]
        cc = np.where(mask.any(axis=0))[0]
        crop = crop[rr.min():rr.max()+1, cc.min():cc.max()+1]
    # pad to square (white background ~1.0)
    h, w = crop.shape
    s = max(h, w) + 2*pad
    pad_t = (s - h) // 2
    pad_b = s - h - pad_t
    pad_l = (s - w) // 2
    pad_r = s - w - pad_l
    sq = np.pad(crop, ((pad_t, pad_b), (pad_l, pad_r)), constant_values=1.0)
    # resize to out x out and invert so digit is bright
    img28 = resize(sq, (out, out), anti_aliasing=True)
    img28 = 1.0 - img28  # digit -> high, background -> low
    img28 = np.clip(img28, 0, 1)
    return img28

digits = [crop_norm(gray, b) for b in boxes]

# show a grid
n = len(digits)
assert n > 0, 'No components found to crop.'
cols = min(8, n)
rows = int(np.ceil(n / cols))
fig, axes = plt.subplots(rows, cols, figsize=(1.5*cols, 1.5*rows))
axes = np.array(axes).reshape(rows, cols)
k = 0
for r in range(rows):
    for c in range(cols):
        ax = axes[r, c]
        ax.axis('off')
        if k < n:
            ax.imshow(digits[k], cmap='gray', vmin=0, vmax=1)
            k += 1
plt.suptitle(f'Extracted digits: {n}', y=0.98)
plt.tight_layout()

Classify the extracted 28×28 crops with the MNIST CNN; prints predicted sequence. If TensorFlow isn’t installed on this Python (e.g., 3.13), it skips classification.

In [ ]:
# Ensure HW_1/src is on sys.path so `from model import ...` works
import sys
from pathlib import Path

def add_src_to_path():
    bases = [Path.cwd(), *Path.cwd().parents]
    for base in bases:
        for rel in ('src', 'HW_1/src'):
            cand = base / rel
            if (cand / 'model.py').exists():
                if str(cand) not in sys.path:
                    sys.path.insert(0, str(cand))
                print('Using src path:', cand)
                return True
    print('model.py not found. Check that HW_1/src/model.py exists.')
    return False

ok = add_src_to_path()
if ok:
    try:
        from model import load_or_train_default, predict_digits
        print('Import OK: load_or_train_default, predict_digits')
    except Exception as e:
        print('Import still failed:', e)

# Now re-run the previous classification cell.

In [ ]:
# Rebuild boxes and 28x28 digit crops
from pathlib import Path
import numpy as np
from skimage import io
from skimage.color import rgb2gray
from skimage.filters import gaussian, threshold_otsu
from skimage.transform import resize
from scipy import ndimage as ndi

# load image if needed
if 'img' not in globals():
    IMAGES_DIR = globals().get('IMAGES_DIR', Path.cwd() / 'images1')
    img = io.imread(sorted(IMAGES_DIR.glob('*.jpg'))[0])

gray = rgb2gray(img)
mask = gaussian(gray, 1.0) < threshold_otsu(gaussian(gray, 1.0))

labels, _ = ndi.label(mask)
slices = ndi.find_objects(labels)
areas = np.bincount(labels.ravel())
min_size = 150

boxes = []
for i, slc in enumerate(slices, start=1):
    if slc is None or areas[i] < min_size:
        continue
    r0, r1 = slc[0].start, slc[0].stop
    c0, c1 = slc[1].start, slc[1].stop
    boxes.append((r0, c0, r1, c1))

# simple reading order
boxes.sort(key=lambda b: (b[0], b[1]))

def to28(g, b, out=28, pad=2):
    r0, c0, r1, c1 = b
    crop = g[r0:r1, c0:c1]
    h, w = crop.shape
    s = max(h, w) + 2*pad
    pt, pb = (s - h)//2, s - h - (s - h)//2
    pl, pr = (s - w)//2, s - w - (s - w)//2
    sq = np.pad(crop, ((pt, pb), (pl, pr)), constant_values=1.0)  # white bg
    img28 = resize(sq, (out, out), anti_aliasing=True)
    return np.clip(1.0 - img28, 0, 1)  # invert: digit bright

digits = [to28(gray, b) for b in boxes]
print(f'Rebuilt digits: {len(digits)}')

Classify the 28×28 crops with the MNIST CNN; prints the predicted sequence

In [ ]:
# python
# Extract 28x28 digits from the first JPG in `images1` and optionally classify
from pathlib import Path
import numpy as np
from skimage import io, color, filters, transform
from scipy import ndimage as ndi
import importlib.util, sys

def extract_digits(images_dir='images1', min_size=150, out=28, pad=2):
    p = Path(images_dir)
    img_file = next(p.glob('*.jpg'), None)
    assert img_file is not None, f'No .jpg in {p}'
    img = io.imread(img_file)
    gray = color.rgb2gray(img)
    blur = filters.gaussian(gray, sigma=1.0)
    mask = blur < filters.threshold_otsu(blur)
    labels, _ = ndi.label(mask)
    slices = ndi.find_objects(labels)
    areas = np.bincount(labels.ravel())
    boxes = []
    for i, slc in enumerate(slices, start=1):
        if slc is None or areas[i] < min_size:
            continue
        r0, r1 = slc[0].start, slc[0].stop
        c0, c1 = slc[1].start, slc[1].stop
        boxes.append((r0, c0, r1, c1))
    if not boxes:
        return []
    # reading order
    med_h = max(1, int(np.median([b[2]-b[0] for b in boxes])))
    row_h = max(1, int(0.6 * med_h))
    boxes = sorted(boxes, key=lambda b: (b[0] // row_h, b[1]))
    def to28(g, b):
        r0, c0, r1, c1 = b
        crop = g[r0:r1, c0:c1]
        # trim empty margins
        thr = np.percentile(crop, 80)
        mask = crop < thr
        if mask.any():
            rr = np.where(mask.any(axis=1))[0]
            cc = np.where(mask.any(axis=0))[0]
            crop = crop[rr.min():rr.max()+1, cc.min():cc.max()+1]
        h, w = crop.shape
        s = max(h, w) + 2*pad
        pt = (s - h)//2; pl = (s - w)//2
        sq = np.pad(crop, ((pt, s - h - pt), (pl, s - w - pl)), constant_values=1.0)
        img28 = transform.resize(sq, (out, out), anti_aliasing=True)
        return np.clip(1.0 - img28, 0, 1)
    digits = [to28(gray, b) for b in boxes]
    return digits

# Usage:
digits = extract_digits('images1')
assert len(digits) > 0, 'No digit crops found; check images and dependencies.'
print('Extracted digits:', len(digits))

# Optional: import local model implementation at `HW_1/src/model.py` and classify
model_py = Path.cwd() / 'HW_1' / 'src' / 'model.py'
if model_py.exists():
    spec = importlib.util.spec_from_file_location('cv_hw_model', str(model_py))
    cv_hw_model = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(cv_hw_model)
    try:
        model = cv_hw_model.load_or_train_default(str(Path.cwd() / 'model.h5'))
        patches = (np.array(digits) * 255).astype(np.uint8)
        preds = cv_hw_model.predict_digits(model, patches)
        print('Predicted:', ''.join(map(str, preds.tolist())))
    except Exception as e:
        print('Classification skipped or failed:', type(e).__name__, e)
else:
    print('Local model.py not found at `HW_1/src/model.py`; skipping classification.')

Detects paper corners and warps the sheet to A4; it shows results and keeps warped images for later

In [ ]:
# Detect paper corners and warp to A4; keeps warped_bgr/warped_gray for later steps
from pathlib import Path
import sys, cv2, numpy as np, matplotlib.pyplot as plt

# Ensure local src/ is importable
SRC_DIR = Path.cwd() / 'src'
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

from paper_detect import detect_paper_corners, warp_to_a4  # uses OpenCV

# Pick a sample image (first .jpg in images1)
IMAGES_DIR = Path.cwd() / 'images1'
image_files = sorted(IMAGES_DIR.glob('*.jpg'))
assert image_files, f'No .jpg images found in {IMAGES_DIR}'
img_path = image_files[0]

bgr = cv2.imread(str(img_path))
assert bgr is not None, f'Failed to read {img_path}'

corners = detect_paper_corners(bgr)  # (4,2) [tl,tr,br,bl] or None
if corners is None:
    print('Paper not found in', img_path.name)
else:
    vis = bgr.copy()
    for (x, y) in corners.astype(int):
        cv2.circle(vis, (x, y), 6, (0, 0, 255), -1)  # draw corner

    warped = warp_to_a4(bgr, corners)  # perspective warp to A4 ratio

    # Show original + corners vs warped
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)); axes[0].set_title(f'{img_path.name} + corners'); axes[0].axis('off')
    axes[1].imshow(cv2.cvtColor(warped, cv2.COLOR_BGR2RGB)); axes[1].set_title('Warped (A4)'); axes[1].axis('off')
    plt.tight_layout()

    # Keep for later cells
    warped_bgr = warped
    warped_gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
    print('Corners (tl,tr,br,bl):\n', corners.astype(int))

Binarize the warped sheet, extract components, draw boxes, and prepare 28×28 crops. Insert this as one code cell into

In [ ]:
# Binarize warped sheet, extract components, draw boxes, and prepare 28x28 digit crops
from pathlib import Path
import sys, numpy as np, cv2, matplotlib.pyplot as plt

# Ensure local src/ is importable
SRC_DIR = Path.cwd() / 'src'
if SRC_DIR.exists() and str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

from paper_detect import detect_paper_corners, warp_to_a4
from segments import binarize, extract_components, crop_and_resize

# Ensure we have a warped image from the previous cell; fallback if missing
if 'warped_bgr' not in globals():
    IMAGES_DIR = Path.cwd() / 'images1'
    img_path = next(iter(sorted(IMAGES_DIR.glob('*.jpg'))), None)
    assert img_path is not None, f'No .jpg images in {IMAGES_DIR}'
    bgr0 = cv2.imread(str(img_path)); assert bgr0 is not None
    c = detect_paper_corners(bgr0)
    warped_bgr = warp_to_a4(bgr0, c) if c is not None else bgr0

# 1) Binarize (robust to uneven illumination)
bin_img = binarize(warped_bgr)

# 2) Extract connected components (boxes: x,y,w,h), sorted top->bottom then left->right
boxes = extract_components(bin_img)

# 3) Visualize boxes on warped image and the binary mask
vis = warped_bgr.copy()
for (x, y, w, h) in boxes:
    cv2.rectangle(vis, (x, y), (x+w, y+h), (0, 255, 0), 2)

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)); axes[0].set_title(f'Boxes: {len(boxes)}'); axes[0].axis('off')
axes[1].imshow(bin_img, cmap='gray'); axes[1].set_title('Binary'); axes[1].axis('off')
plt.tight_layout()

# 4) Crop and resize to 28x28 (MNIST-style: digit bright on dark). Keep for later classification.
if boxes:
    patches28 = np.stack([crop_and_resize(warped_bgr, b, 28) for b in boxes], axis=0).astype(np.uint8)
    # Show a small grid
    n = len(patches28); cols = min(8, n); rows = int(np.ceil(n / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(1.4*cols, 1.4*rows))
    axes = np.atleast_2d(axes)
    k = 0
    for r in range(rows):
        for c in range(cols):
            ax = axes[r, c] if rows > 1 or cols > 1 else axes[0, 0]
            ax.axis('off')
            if k < n:
                ax.imshow(patches28[k], cmap='gray', vmin=0, vmax=255); k += 1
    plt.suptitle(f'28×28 digit crops: {n}', y=0.98); plt.tight_layout()
else:
    patches28 = np.empty((0, 28, 28), dtype=np.uint8)

print(f'Components found: {len(boxes)}; patches28 shape: {patches28.shape}')

Classify the 28×28 crops from the previous step and print the predicted digit sequence.

In [ ]:
# Classify 28x28 digit crops (patches28) with the MNIST CNN and print the sequence
from pathlib import Path
import sys, numpy as np, matplotlib.pyplot as plt

# Ensure local src is importable (tries ./src and ./HW_1/src)
for base in (Path.cwd(), Path.cwd() / 'HW_1'):
    cand = base / 'src'
    if (cand / 'model.py').exists() and str(cand) not in sys.path:
        sys.path.insert(0, str(cand))

try:
    from model import load_or_train_default, predict_digits
except Exception as e:
    print('Classification unavailable: cannot import model.py:', e)
    preds = np.array([], dtype=int)
else:
    # Build patches28 if missing (uses warped_bgr from previous cell)
    if 'patches28' not in globals() or patches28.size == 0:
        try:
            from segments import binarize, extract_components, crop_and_resize
            from paper_detect import detect_paper_corners, warp_to_a4
            import cv2
            # Try to create from warped_bgr; otherwise from first image in images1/
            if 'warped_bgr' not in globals():
                IMAGES_DIR = Path.cwd() / 'images1'
                img_path = next(iter(sorted(IMAGES_DIR.glob('*.jpg'))), None)
                assert img_path is not None, f'No .jpg images in {IMAGES_DIR}'
                bgr0 = cv2.imread(str(img_path)); assert bgr0 is not None
                c = detect_paper_corners(bgr0)
                warped_bgr = warp_to_a4(bgr0, c) if c is not None else bgr0
            bin_img = binarize(warped_bgr)
            boxes = extract_components(bin_img)
            patches28 = np.stack([crop_and_resize(warped_bgr, b, 28) for b in boxes], axis=0).astype(np.uint8) if boxes else np.empty((0,28,28), np.uint8)
        except Exception as e:
            print('Could not prepare patches28 automatically:', e)
            patches28 = np.empty((0,28,28), np.uint8)

    if patches28.size == 0:
        print('No digit crops available (patches28 is empty).')
        preds = np.array([], dtype=int)
    else:
        try:
            model_path = Path.cwd() / 'HW_1' / 'model.h5'
            model = load_or_train_default(str(model_path))  # trains briefly if file missing
            preds = predict_digits(model, patches28)
        except Exception as e:
            print('Classification skipped or failed:', e)
            preds = np.array([], dtype=int)

# Report results and quick visualization
if preds.size:
    seq = ''.join(map(str, preds.tolist()))
    print('Predicted sequence:', seq)
    # Show a small grid with predicted labels
    n = len(patches28); cols = min(8, n); rows = int(np.ceil(n / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(1.4*cols, 1.6*rows))
    axes = np.atleast_2d(axes)
    k = 0
    for r in range(rows):
        for c in range(cols):
            ax = axes[r, c] if rows > 1 or cols > 1 else axes[0, 0]
            ax.axis('off')
            if k < n:
                ax.imshow(patches28[k], cmap='gray', vmin=0, vmax=255)
                ax.set_title(str(int(preds[k])), fontsize=10)
                k += 1
    plt.tight_layout()
else:
    print('No predictions to display.')

Process all images in a directory and print “filename: digits” per file.

In [ ]:
# Reload src modules and re-run processing for all images (uses fixed corner refinement)
from pathlib import Path
import sys, importlib

ROOT = Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))  # allow `import src.*`

import src.paper_detect as paper_detect
import src.segments as segments
import src.model as model_mod
import src.pipeline as pipeline

# Reload to pick up latest edits on disk
for m in (paper_detect, segments, model_mod, pipeline):
    importlib.reload(m)

images_dir = ROOT / 'images1'  # change to 'images2' to test the other set
assert images_dir.exists(), f'Missing images directory: {images_dir}'

# Collect files
files = []
for ext in ('*.jpg','*.jpeg','*.png','*.JPG','*.PNG'):
    files += sorted(images_dir.glob(ext))
assert files, f'No images found in {images_dir}'

# Load/train model (skips if TF unavailable)
model_path = ROOT / 'HW_1' / 'model.h5'
try:
    model = model_mod.load_or_train_default(str(model_path))
except Exception:
    print('TensorFlow/Keras unavailable; proceeding without classification.')
    model = None

# Run pipeline per file with safety guard
for p in files:
    try:
        print(pipeline.process_image(p, model, model_path))
    except Exception as e:
        print(f'{p.name}: [error: {type(e).__name__}]')

 Prints “filename: digits”, and saves overlays/grids and a results file to HW_1/results.

In [ ]:
# Process all images, print "filename: digits", and save overlays/grids + results.txt to HW_1/results
from pathlib import Path
import sys, os, cv2, numpy as np, matplotlib.pyplot as plt

# Make src importable (tries ./src and ./HW_1/src)
ROOT = Path.cwd()
for cand in (ROOT / 'src', ROOT / 'HW_1' / 'src'):
    if (cand / '__init__.py').exists() and str(cand) not in sys.path:
        sys.path.insert(0, str(cand))

# Imports from src
try:
    from paper_detect import detect_paper_corners, warp_to_a4
    from segments import binarize, extract_components, crop_and_resize
    from model import load_or_train_default, predict_digits
except Exception as e:
    raise RuntimeError('Failed to import from src/. Ensure HW_1/src is present.') from e

# Config
images_dir = ROOT / 'images1'  # change to ROOT / 'images2' to run the other set
out_dir = ROOT / 'HW_1' / 'results'
out_dir.mkdir(parents=True, exist_ok=True)

# Collect input files
files = []
for ext in ('*.jpg','*.jpeg','*.png','*.JPG','*.PNG'):
    files += sorted(images_dir.glob(ext))
assert files, f'No images found in {images_dir}'

# Load model (skips classification if TF unavailable)
try:
    model_path = ROOT / 'HW_1' / 'model.h5'
    model = load_or_train_default(str(model_path))
except Exception:
    print('TensorFlow/Keras unavailable; proceeding without classification.')
    model = None

lines = []
for p in files:
    bgr = cv2.imread(str(p))
    if bgr is None:
        msg = f'{p.name}: [unreadable]'
        print(msg); lines.append(msg); continue

    # Detect and warp
    corners = None
    try:
        corners = detect_paper_corners(bgr)
    except Exception:
        corners = None
    warped = warp_to_a4(bgr, corners) if corners is not None else bgr

    # Binarize and components
    bin_img = binarize(warped)
    boxes = extract_components(bin_img)

    # Prepare patches and classify (optional)
    preds = None
    if boxes:
        patches28 = np.stack([crop_and_resize(warped, b, 28) for b in boxes], axis=0).astype(np.uint8)
        if model is not None:
            try:
                preds = predict_digits(model, patches28)
            except Exception:
                preds = None

    # Compose output line
    if preds is None:
        if boxes:
            msg = f'{p.name}: components={len(boxes)} (classification disabled)'
        else:
            msg = f'{p.name}: []'
    else:
        seq = ''.join(map(str, preds.tolist()))
        msg = f'{p.name}: {seq}'
    print(msg); lines.append(msg)

    # Save overlay (boxes + optional labels)
    try:
        vis = warped.copy()
        for i, (x, y, w, h) in enumerate(boxes):
            cv2.rectangle(vis, (x, y), (x+w, y+h), (0, 255, 0), 2)
            if preds is not None:
                cv2.putText(vis, str(int(preds[i])), (x, max(0, y-5)),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2, cv2.LINE_AA)
        cv2.imwrite(str(out_dir / f'{p.stem}_overlay.jpg'), vis)
    except Exception:
        pass

    # Save grid of 28x28 crops
    try:
        if boxes:
            n = len(boxes); cols = min(8, n); rows = int(np.ceil(n / cols))
            fig, axes = plt.subplots(rows, cols, figsize=(1.4*cols, 1.4*rows))
            axes = np.atleast_2d(axes)
            k = 0
            for r in range(rows):
                for c in range(cols):
                    ax = axes[r, c] if rows > 1 or cols > 1 else axes[0, 0]
                    ax.axis('off')
                    if k < n:
                        ax.imshow(patches28[k], cmap='gray', vmin=0, vmax=255)
                        if preds is not None:
                            ax.set_title(str(int(preds[k])), fontsize=9)
                        k += 1
            plt.tight_layout()
            fig.savefig(out_dir / f'{p.stem}_grid.jpg', dpi=150)
            plt.close(fig)
    except Exception:
        pass

# Save results file
(results_txt := out_dir / 'results.txt').write_text('\n'.join(lines))
print(f'Saved results to: {results_txt}')
print(f'Overlays and grids in: {out_dir}')

In [ ]:
# Evaluate images1 and images2 (if present), print per-file results, and save per-set CSV summaries to HW_1/results
from pathlib import Path
import sys, importlib, csv

ROOT = Path.cwd()
# Make 'src' importable as a package
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

import src.pipeline as pipeline
import src.model as model_mod

# Reload to pick up latest src edits (corner fixes, etc.)
importlib.reload(pipeline)
importlib.reload(model_mod)

sets = [ROOT / 'images1', ROOT / 'images2']
sets = [d for d in sets if d.exists()]

assert sets, 'No images1/ or images2/ directories found.'

# Load or train model once (skip classification if TF unavailable)
model_path = ROOT / 'HW_1' / 'model.h5'
try:
    model = model_mod.load_or_train_default(str(model_path))
except Exception:
    print('TensorFlow/Keras unavailable; proceeding without classification.')
    model = None

out_dir = ROOT / 'HW_1' / 'results'
out_dir.mkdir(parents=True, exist_ok=True)

def collect_files(d):
    files = []
    for ext in ('*.jpg','*.jpeg','*.png','*.JPG','*.PNG'):
        files += sorted(d.glob(ext))
    return files

for d in sets:
    files = collect_files(d)
    if not files:
        print(f'No images in {d}')
        continue

    print(f'\n=== Processing: {d.name} ({len(files)} images) ===')
    lines = []
    stats = {'total': len(files), 'unreadable': 0, 'no_paper': 0, 'empty': 0, 'classified': 0, 'components_only': 0}

    for p in files:
        s = pipeline.process_image(p, model, model_path)
        print(s)
        lines.append(s)

        # crude status parsing
        if '[unreadable]' in s:
            stats['unreadable'] += 1
        elif '[paper not found]' in s:
            stats['no_paper'] += 1
        elif s.endswith(': []'):
            stats['empty'] += 1
        elif 'components=' in s:
            stats['components_only'] += 1
        else:
            stats['classified'] += 1

    # Save CSV summary
    csv_path = out_dir / f'results_{d.name}.csv'
    with csv_path.open('w', newline='') as f:
        w = csv.writer(f)
        w.writerow(['filename', 'result'])
        for s in lines:
            if ': ' in s:
                name, res = s.split(': ', 1)
            else:
                name, res = s, ''
            w.writerow([name, res])

    # Print brief summary
    print(f'\nSummary for {d.name}:')
    print(f"  total={stats['total']}, unreadable={stats['unreadable']}, no_paper={stats['no_paper']}, empty={stats['empty']},")
    print(f"  components_only={stats['components_only']}, classified={stats['classified']}")
    print(f'Saved CSV: {csv_path}')

In [ ]:
# Failure analysis: show up to 3 hard cases (paper not found or no components) from images1/images2
from pathlib import Path
import sys, cv2, numpy as np, matplotlib.pyplot as plt

# Make src importable
ROOT = Path.cwd()
for cand in (ROOT / 'src', ROOT / 'HW_1' / 'src'):
    if (cand / '__init__.py').exists() and str(cand) not in sys.path:
        sys.path.insert(0, str(cand))

from paper_detect import detect_paper_corners, warp_to_a4
from segments import binarize, extract_components

def debug_show(p: Path):
    bgr = cv2.imread(str(p))
    if bgr is None:
        print('Unreadable:', p.name); return
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    v = np.median(gray); lo, hi = int(max(0, 0.66*v)), int(min(255, 1.33*v))
    edges = cv2.Canny(gray, lo, hi)

    corners = None
    try:
        corners = detect_paper_corners(bgr)
    except Exception:
        corners = None

    vis0 = bgr.copy()
    if corners is not None:
        for (x, y) in corners.astype(int):
            cv2.circle(vis0, (x, y), 6, (0, 0, 255), -1)

    warped = warp_to_a4(bgr, corners) if corners is not None else bgr
    bin_img = binarize(warped)
    boxes = extract_components(bin_img)

    vis_w = warped.copy()
    for (x, y, w, h) in boxes:
        cv2.rectangle(vis_w, (x, y), (x+w, y+h), (0, 255, 0), 2)

    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    axes[0].imshow(cv2.cvtColor(vis0, cv2.COLOR_BGR2RGB)); axes[0].set_title(f'{p.name} + corners'); axes[0].axis('off')
    axes[1].imshow(edges, cmap='gray'); axes[1].set_title('Edges'); axes[1].axis('off')
    axes[2].imshow(cv2.cvtColor(vis_w, cv2.COLOR_BGR2RGB)); axes[2].set_title(f'Warped + boxes ({len(boxes)})'); axes[2].axis('off')
    plt.tight_layout()
    status = 'OK' if corners is not None and len(boxes) > 0 else ('no_paper' if corners is None else 'no_components')
    print(f'{p.name}: {status}')

def collect_files(d):
    files = []
    for ext in ('*.jpg','*.jpeg','*.png','*.JPG','*.PNG'):
        files += sorted(d.glob(ext))
    return files

candidates = []
for d in (ROOT / 'images1', ROOT / 'images2'):
    if d.exists():
        candidates += collect_files(d)

shown = 0
for p in candidates:
    # quick probe to pick failures
    bgr = cv2.imread(str(p))
    if bgr is None:
        continue
    try:
        c = detect_paper_corners(bgr)
        warped = warp_to_a4(bgr, c) if c is not None else bgr
        boxes = extract_components(binarize(warped))
        is_fail = (c is None) or (len(boxes) == 0)
    except Exception:
        is_fail = True
    if is_fail:
        debug_show(p); shown += 1
        if shown >= 3:
            break

if shown == 0 and candidates:
    print('No obvious failures found; showing two successes for reference.')
    for p in candidates[:2]:
        debug_show(p)